Following Foxtrot's script https://www.kaggle.com/zygmunt/rossmann-store-sales/predict-sales-with-pandas-py but including splitting on SchoolHoliday.

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib
import os
from ggplot import *

In [74]:
foutput = 'group_medians_output.csv'

In [39]:
#train = pd.read_csv(os.path.join('data','train.csv'), parse_dates=['Date'])
train = pd.read_csv(os.path.join('data','train.csv'), parse_dates=['Date'], converters = {'StateHoliday': str})
test = pd.read_csv(os.path.join('data','test.csv'), parse_dates=['Date'])
store = pd.read_csv(os.path.join('data','store.csv'))

In [40]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1,1,0,0
1,2,3,4,2015-09-17,1,1,0,0
2,3,7,4,2015-09-17,1,1,0,0
3,4,8,4,2015-09-17,1,1,0,0
4,5,9,4,2015-09-17,1,1,0,0


In [41]:
train.StateHoliday.value_counts()

0    986159
a     20260
b      6690
c      4100
dtype: int64

In [42]:
# remove rows where Sales == 0 (mostly days when closed, but 54 days when open), 
# StateHoliday == b or c (not present in test set)
train2 = train[~((train['Sales'] == 0) | train['StateHoliday'].isin(['b','c'])) ]

In [43]:
len(train2.index)

844122

In [44]:
# replace NaN's for Open in test (for store 622) with 0.
test.loc[test['Open'].isnull(), 'Open'] = 0

In [63]:
# columns to group by
groups = ['Store', 'DayOfWeek', 'Promo', 'SchoolHoliday']

In [64]:
medians = train2.groupby(groups)['Sales'].median()

In [65]:
medians.head()

Store  DayOfWeek  Promo  SchoolHoliday
1      1          0      0                3965.0
                         1                4260.0
                  1      0                5654.5
                         1                6148.0
       2          0      0                3773.0
Name: Sales, dtype: float64

In [66]:
medians = medians.reset_index()

In [67]:
medians.head()

,Store,DayOfWeek,Promo,SchoolHoliday,Sales
0,1,1,0,0,3965.0
1,1,1,0,1,4260.0
2,1,1,1,0,5654.5
3,1,1,1,1,6148.0
4,1,2,0,0,3773.0


In [68]:
test2 = pd.merge(test, medians, on = groups, how='left')

In [69]:
assert(len(test2) == len(test))

In [70]:
test2['Sales'].isnull().sum()

5796

In [71]:
test2.loc[test2.Open==0, 'Sales'] = 0
assert( test2.Sales.isnull().sum() == 0 )

In [76]:
test2[['Id','Sales']].to_csv(foutput, index=False)